## Innovation Maturity Model (IMM)

The **Innovation Maturity Model (IMM)** is a framework that helps organizations evaluate and improve their innovation capacity and processes. The model provides a structured approach to assessing an organization's innovation maturity by analyzing various dimensions of innovation, such as strategy, culture, processes, leadership, technology, and collaboration. The purpose of the model is to identify the organization's level of innovation maturity and outline the steps necessary to further enhance innovation and leverage it strategically.

### Key Dimensions of the Innovation Maturity Model:

- **Innovation Strategy**: Does the organization have a clear vision and strategy for innovation, and how broadly is it shared?
- **Innovation Culture**: Is innovation actively promoted in the organizational culture, and are employees encouraged to introduce and experiment with new ideas?
- **Leadership & Support**: To what extent does the organization’s leadership support and encourage innovation initiatives?
- **Process and Structure**: Are there structured processes in place for developing, testing, and implementing new ideas?
- **Use of Technology**: To what extent does the organization effectively utilize technology to enable and accelerate innovation?
- **Knowledge Sharing and Collaboration**: How well is knowledge shared within teams and departments to foster innovation?
- **Customer-Centric Innovation**: Are customer needs and feedback incorporated into the innovation process?
- **Risk Management**: How does the organization manage the risks associated with innovation projects?
- **Innovation Budget**: Is there sufficient budget allocated to support innovation?
- **Speed of Innovation**: How quickly can the organization turn new ideas into realistic and executable projects?

### Innovation Maturity Levels:

The IMM distinguishes between various levels of maturity, often categorized into 4 or 5 stages, from "beginner" to "innovation leader":

1. **Ad-hoc**: Innovation is sporadic and unstructured, with no clear process or strategy.
2. **Repeatable**: Innovation occurs more frequently but remains largely dependent on individual initiatives and is not widespread.
3. **Defined**: Innovation processes are established and consistently applied. There is broader involvement across teams and departments.
4. **Managed**: Innovation is integrated into the business strategy, with clear KPIs and management processes in place to drive innovation.
5. **Optimized**: The organization is an innovation leader, with continuous improvement supported by advanced technologies and a strong culture of creativity and collaboration.

### Purpose of the IMM:

The purpose of the IMM is to help organizations understand where they stand on this scale and identify actions to elevate their innovation capabilities. It serves not only as a diagnostic tool but also as a roadmap for strategic improvement.


##### CONFIG SETTINGS

In [27]:
#!pip install numpy==1.23.5 langchain-openai==0.1.6 markdown2==2.4.13

##### IMPORTS

In [28]:
import json
import markdown2
import numpy as np
import pandas as pd
import os
import re
from collections import defaultdict
from datetime import datetime, timedelta
from langchain.chains import SequentialChain, LLMChain
from langchain_openai import ChatOpenAI
from langchain_openai.llms import OpenAI
from langchain_core.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate, PromptTemplate

##### INITIALIZATIONS

In [29]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

##### SPECIFIC PARAMETERS

In [30]:
DATA_FOLDER = 'data'
TEMPLATES_FOLDER = 'templates'
QUESTIONNAIRE = "imm_questionaire.csv"
report_name = "Innovation-Scan Report.md"

##### PATHS

In [31]:
# Input paths.
input_questionnaire_file_path = os.path.join(DATA_FOLDER, QUESTIONNAIRE)

# Output paths.
output_report_file_path = os.path.join(os.curdir, report_name)

##### SPECIFIC FUNCTIONS

In [32]:
def transform_file(file_path: str) -> str:
    """
    Transform the CSV file by transposing the columns and convert it to JSON.
    """
    # Read the CSV file.
    df_full = pd.read_csv(file_path)

    # The text column names.
    column_names = df_full.columns[0:]

    # The result.
    result = {}

    # Add the data for each question.
    questions_data = []
    for column in column_names:
        question_data = df_full[column].tolist()
        questions_data.append({"vraag": column, "antwoord": question_data})

    result['vragen_en_antwoorden'] = questions_data

    # Convert the result to JSON.
    return json.dumps(result, ensure_ascii=False, indent=4)

In [33]:
def get_template(template_name: str, section: str = None) -> str:
    """
    Returns the contents of the specific template.
    """
    if section is None:
        template_file_path = os.path.join(os.path.join(TEMPLATES_FOLDER, f'{template_name}.md'))
    else:
        template_file_path = os.path.join(os.path.join(TEMPLATES_FOLDER, section), f'{template_name}.md')

    # Load the data.
    with open(template_file_path, 'r', encoding='utf-8') as file:
        contents = file.read()

    return contents

In [34]:
def generate_analysis(questionnaire: dict, system_template: str, human_template: str) -> str:
    system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
    human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
    chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

    request = chat_prompt.format_prompt(
        questionnaire=json.dumps(questionnaire, ensure_ascii=False, indent=4),
    ).to_messages()

    # Define the LLM.
    chat = ChatOpenAI(temperature=0.7, max_tokens=4096, model="gpt-4o-2024-05-13", openai_api_key=OPENAI_API_KEY)
    result = chat(request)

    return result.content

In [35]:
def generate_recommendations(analysis: str, system_template: str, human_template: str) -> str:
    system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
    human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
    chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

    request = chat_prompt.format_prompt(
        analysis=analysis,
    ).to_messages()

    # Define the LLM.
    chat = ChatOpenAI(temperature=0.7, max_tokens=4096, model="gpt-4o-2024-05-13", openai_api_key=OPENAI_API_KEY)
    result = chat(request)

    return result.content

##### MAIN

In [36]:
# Transpose the columns of the questionnaire.
completed_questionnaire_content = transform_file(input_questionnaire_file_path)

In [37]:
# Analysis.
analysis = generate_analysis(
    questionnaire=completed_questionnaire_content,
    system_template=get_template('system', 'analysis'),
    human_template=get_template('human', 'analysis')
)

In [38]:
# Recommendations.
recommendations = generate_recommendations(
    analysis=analysis,
    system_template=get_template('system', 'recommendations'),
    human_template=get_template('human', 'recommendations')
)

In [39]:
# Define the LLM.
model = ChatOpenAI(temperature=0.7, max_tokens=4096, model="gpt-4o-2024-05-13", openai_api_key=OPENAI_API_KEY)

In [40]:
# Define the initial check prompt.
initial_check_prompt = PromptTemplate(
    input_variables=[
        "analysis",
        "recommendations"
    ],
    template=get_template('prompt', 'check')
)

In [41]:
# Combined raw content.
combined_raw_content = {
    "analysis": analysis,
    "recommendations": recommendations
}

In [42]:
# Run the initial check.
initial_check_prompt_filled = initial_check_prompt.format(**combined_raw_content)
initial_check = model.invoke(input=initial_check_prompt_filled)

In [43]:
# Define the prompts.
consistency_prompt = PromptTemplate(input_variables=["section_content", "feedback"], template=get_template('consistency', 'refinement'))
coherence_prompt = PromptTemplate(input_variables=["consistency_content", "feedback"], template=get_template('coherence', 'refinement'))
language_prompt = PromptTemplate(input_variables=["coherence_content", "feedback"], template=get_template('language', 'refinement'))
fact_checking_prompt = PromptTemplate(input_variables=["language_content", "feedback"], template=get_template('fact_checking', 'refinement'))
clarity_readability_prompt = PromptTemplate(input_variables=["fact_checking_content", "feedback"], template=get_template('clarity_readability', 'refinement'))
relevance_conciseness_prompt = PromptTemplate(input_variables=["clarity_readability_content", "feedback"], template=get_template('relevance_conciseness', 'refinement'))

In [44]:
# Define the LLM.
llm = ChatOpenAI(temperature=0.7, max_tokens=4096, model="gpt-4o-2024-05-13", openai_api_key=OPENAI_API_KEY)

In [45]:
# Define individual chains for each check.
consistency_chain = LLMChain(llm=llm, prompt=consistency_prompt, output_key="consistency_content")
coherence_chain = LLMChain(llm=llm, prompt=coherence_prompt, output_key="coherence_content")
language_chain = LLMChain(llm=llm, prompt=language_prompt, output_key="language_content")
fact_checking_chain = LLMChain(llm=llm, prompt=fact_checking_prompt, output_key="fact_checking_content")
clarity_readability_chain = LLMChain(llm=llm, prompt=clarity_readability_prompt, output_key="clarity_readability_content")
relevance_conciseness_chain = LLMChain(llm=llm, prompt=relevance_conciseness_prompt, output_key="refined_section")

In [46]:
# Combine into a sequential chain for refinement.
refinement_chain = SequentialChain(
    chains=[consistency_chain, coherence_chain, language_chain, fact_checking_chain, clarity_readability_chain, relevance_conciseness_chain],
    input_variables=["section_content", "feedback"],
    output_variables=["refined_section"]
)

In [47]:
# The refined content.
refined_analysis = refinement_chain.run(section_content=analysis, feedback=initial_check)
refined_recommendations = refinement_chain.run(section_content=recommendations, feedback=initial_check)

In [63]:
# Innovation-Scan report.
report = get_template('report').format(
    analysis=refined_analysis,
    recommendations=refined_recommendations
)

# Write the report.
with open(output_report_file_path, 'w', encoding='utf-8') as file:
    file.write(report)